In [32]:
import numpy as np
import math
import matplotlib.pyplot as plt
from akorn import *
%load_ext autoreload
%autoreload 2

%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
%load_ext autoreload
%autoreload 2
# import numpy as np
# import math
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr, data



utils = importr('utils')
base = importr('base')
glmgen = importr('glmgen')
%matplotlib qt
def pathology(x):
    if x < 0.5:
        return 0
    elif x <= 0.5001:
        return 100*(x - 0.5)
    else:
        return 100*0.001
std = 0.3
var = std**2

mse_akorn = []
mse_tf = []



def doppler(x, epsilon):
    return np.sin(2 * np.pi * (1 + epsilon) / (x + epsilon))

from util import piecewise_linear

print("Done!")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Done!


In [34]:
mse_akorn = []
mse_akorn05 = []
mse_akorn025 = []
mse_akorn0125 = []
mse_tf = []

# Ts = [100, 5000, 100_000, 200_000, 500_000]
Ts = [50, 100, 200, 500, 1000, 2500, 5000]
# # Ts = [10_000, 20_000, 40_000]
# # Ts = [1000]
# Ts = [int(1000*math.e**(i/5)) for i in range(0, 13)]
# # Ts = [80_000, 160_000]
# # Ts = [10_000]
# # Ts = [100]
runs_per_T = 10

from util import piecewise_linear
    
for T in Ts:
    X = np.linspace(0, T, T)/T 

    G = [piecewise_linear([1,-1,3, -3, 12, -2, -3, 1],np.array([0.25, 0.5, 0.6, 0.8, 1.2, 1.3, 1.7, 1.9])/2, x, 0) for x in X]
    G = G/np.max(G)

    mses_T_tf = []
    mses_T_akorn = []

    for l in range(runs_per_T):
        print("Run number " + str(l))
        Y = (G + np.random.normal(0, std, T))

        ak = AKORN(X, Y, var)
        ak.train()
        fv = ak.preds


        mses_T_akorn.append(((fv - G)**2).mean())


        Xr = robjects.FloatVector(X)


        Yr = robjects.FloatVector(Y)
        tf = glmgen.trendfilter(Xr, Yr, k = 1)
        rcode1 = "predict(%s)" %(tf.r_repr())
        tf_fit = np.array(robjects.r(rcode1)).T

        best_mse = float('inf')
        best_tf_fit = None
        for j in tf_fit:
        #     print("hey")
            tf_mse = ((j - G)**2).mean()
            if tf_mse < best_mse:
                best_mse = tf_mse
                best_tf_fit = j
        mses_T_tf.append(best_mse)

    #     Z_tf.append(best_tf_fit)
    mse_tf.append(mses_T_tf)
    mse_akorn.append(mses_T_akorn)

avg_mses_akorn = [np.array(mse_T).mean() for mse_T in mse_akorn]
avg_mses_tf = [np.array(mse_T).mean() for mse_T in mse_tf]
print("Done!")

Run number 0
lr =  0.03907893139572911
run
new akorn
0
new akorn
0
Run number 1
lr =  0.03907893139572911
run
new akorn
0
new akorn
0
Run number 2
lr =  0.03907893139572911
run
new akorn
0
new akorn
0
Run number 3
lr =  0.03907893139572911
run
new akorn
0
new akorn
0
Run number 4
lr =  0.03907893139572911
run
new akorn
0
new akorn
0
Run number 5
lr =  0.03907893139572911
run
new akorn
0
new akorn
0
Run number 6
lr =  0.03907893139572911
run
new akorn
0
new akorn
0
Run number 7
lr =  0.03907893139572911
run
new akorn
0
new akorn
0
Run number 8
lr =  0.03907893139572911
run
new akorn
0
new akorn
0
Run number 9
lr =  0.03907893139572911
run
new akorn
0
new akorn
0
Run number 0
lr =  0.037444597621460844
run
new akorn
0
new akorn
0
Run number 1
lr =  0.037444597621460844
run
new akorn
0
new akorn
0
Run number 2
lr =  0.037444597621460844
run
new akorn
0
new akorn
0
Run number 3
lr =  0.037444597621460844
run
new akorn
0
new akorn
0
Run number 4
lr =  0.037444597621460844
run
new akorn
0
ne

In [35]:
from sklearn.linear_model import LinearRegression
akorn_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(avg_mses_akorn))
tf_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(np.array(avg_mses_tf)))

In [39]:
def quick_round_str(z):
    return str(round(z, 5))

plt.clf()
from util import subplots_centered
T = 1000
X = np.linspace(0, T, T)/T 
G = [piecewise_linear([1,-1,3, -3, 12, -2, -3, 1],np.array([0.25, 0.5, 0.6, 0.8, 1.2, 1.3, 1.7, 1.9])/2, x, 0) for x in X]
# G = doppler(abs(1 - X), 0.38)
f, ax = subplots_centered(nrows=2, ncols=2, figsize=(10,7), nfigs=3)
ax0 = ax[0]
ax1 = ax[1]
ax1.yaxis.set_visible(False)
ax2 = ax[2]
f.suptitle("Rates for Piecewise Linear Ground Truth with Small Elements, ub = False")
ax0.set_xlabel("n")
ax0.set_ylabel("MSE")
# ax1.set_xlabel("n")
# ax1.set_ylabel("MSE")
ax0.set_ylim([10**(-4), 10**(0)])
ax1.set_ylim([10**(-4), 10**(0)])
ax1.grid(True)
ax1.grid(True)
# plt.loglog(Ts, [t**(-1) for t in Ts])
# plt.loglog(Ts, [t**(-4/5) for t in Ts], label = "$n^{-4/5}$")
ax0.loglog(Ts, avg_mses_akorn, marker = 'o', label = "akorn, rate = " + quick_round_str(akorn_ref.coef_[0]), color = "orange")
# ax0.title.set_text("Least Squares Slope = " + str(akorn_ref.coef_[0]))
ax0.loglog(Ts, avg_mses_tf, marker = 'o', label = "oracle trend filtering, rate = " +  quick_round_str(tf_ref.coef_[0]))
# ax1.title.set_text("Least Squares Slope = " + str(tf_ref.coef_[0]))

ax2.title.set_text("Ground truth")
ax2.plot(X, G, color = "black")
f.legend()

In [41]:
for T in Ts:
    X = np.linspace(0, T, T)/T 

    G = [piecewise_linear([1,-1,3, -3, 12, -2, -3, 1],np.array([0.25, 0.5, 0.6, 0.8, 1.2, 1.3, 1.7, 1.9])/2, x, 0) for x in X]
#     G = G/np.max(G)
    plt.plot(X, G)
    plt.pause(0.1)